In [1]:
#BASIC LOAD
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 15)


csv_file_path = "csvExports/point_data_2023-12-20_15-08-31.csv"
point_df = pd.read_csv(csv_file_path)

csv_file_path = "csvExports/curve_data_2023-11-05_19-38-07.csv"
curve_df = pd.read_csv(csv_file_path)


In [ ]:
#create a new roastNumber value, probably don't need this now..
point_df['roastNumber'] = point_df.index
point_df['roastNumber'] = point_df['roastNumber'].astype(int)


curve_df = curve_df.merge(point_df[['roastName', 'roastNumber']], on='roastName', how='left')
try:
    curve_df['roastNumber'] = curve_df['roastNumber'].astype(np.int64)
except:
    print (curve_df[curve_df['roastNumber'].isna()])
    curve_df['roastNumber'] = curve_df['roastNumber'].fillna(0).astype(np.int64)

display(curve_df[['roastName','roastNumber']])

#getting the roast number of a specific roast. was going to use to this to filter early roasts out
roast_name = "#34 Kenya - Soak and watch"
roast_number = curve_df.loc[curve_df['roastName'] == roast_name, 'roastNumber'].values[0]
print(roast_number)


In [2]:
#replace NaN values with 0 for early bullet roasts
nan_roast_names = curve_df.groupby('roastName')['ibts2ndDerivative'].apply(lambda x: x.isnull().all()).loc[lambda x: x].index.tolist()
print(nan_roast_names)
curve_df.loc[curve_df['roastName'].isin(nan_roast_names), 'ibts2ndDerivative'] = 0

#fill all ibtsDerivative NAN values with 0 (usually just the first 1-2 rows, pretty negligible)
curve_df['ibts2ndDerivative'] = curve_df['ibts2ndDerivative'].fillna(0)

#fill all ibtsDerivative values now (usually just the first row and negligible)
curve_df['ibtsDerivative'] = curve_df['ibtsDerivative'].fillna(0)

['# 25 Kenya Mix 350/140', '#11 Ethiopia Agaro Nano Challa w/ Rob', '#12 PNG Bariod', '#13 Kenya Nyeri Githiru AA', '#17 Ethiopia Limu Musa  Soak 9/9/20', '#18 Eithopian DP Shakios Korommii with sm new DP mix', '#19 Etihopia Limu', '#20 Ethiopia Limu - Fan Rubbing', '#21 Ethiopia Genji Challa', '#22 Ethiopia Gengi Challa DP', '#22 Guat Xinabajul Chchu - typical', '#23 PNG Barioda Estate 1', '#24 PNG Barioda Estate 2nd Actual', '#27 Guat ', '#28 Guat 245PH no crash', '#29 Genji Challa DP - Good', '#30 Burrundi ', '#31 Ethiopia Lima Musu washed and crashed', '#32 PNG / Guat for Tara', '#33 Kenya Gakuyu-Ini AA', '#7 Ethopian - no fan, applied reading, meh', '1/8 Guat Xinabajul Cuchu', '10 Peru with Rob', '2nd Ethopia Extended Dev', '9 Nica DP with Rob', 'Ethiopia #4  w/ marcel', 'Ethiopian 5th - Hot Start', 'First BBP attempt - CItyish', 'First Post-seasoning']


In [17]:
#check the values for a specific roast
roast_name = "#134 - Crown Jewel Ethiopian"
display (curve_df[curve_df['roastName'] == roast_name][['beanDerivative','roastName', 'indexTime']])

#fill all other values now (usually just the first 1-2 rows, pretty negligible)
#curve_df['ibtsDerivative'] = curve_df['ibtsDerivative'].fillna(0)


,beanDerivative,roastName,indexTime
159338,6.8,#134 - Crown Jewel Ethiopian,0
159339,6.8,#134 - Crown Jewel Ethiopian,1
159340,7.5,#134 - Crown Jewel Ethiopian,2
159341,8.5,#134 - Crown Jewel Ethiopian,3
159342,9.6,#134 - Crown Jewel Ethiopian,4
...,...,...,...
160670,9.4,#134 - Crown Jewel Ethiopian,1332
160671,9.4,#134 - Crown Jewel Ethiopian,1333
160672,9.3,#134 - Crown Jewel Ethiopian,1334
160673,9.7,#134 - Crown Jewel Ethiopian,1335


In [3]:
# print max beanDerivative and indexTime for each Roast
for name, group in curve_df.groupby('roastName'):
    max_bean_derivative = group['beanDerivative'].max()
    max_index_time = group.loc[group['beanDerivative'].idxmax(), 'indexTime']
    print(f"Roast Name: {name}, Max beanDerivative: {max_bean_derivative}, Index Time: {max_index_time}")
  

Roast Name: # 25 Kenya Mix 350/140, Max beanDerivative: 19.3, Index Time: 286
Roast Name: #100 Columbia Hulia double ferment - heater error do not use, Max beanDerivative: 18.4, Index Time: 358
Roast Name: #101 El Salvador Siberia Washed w/ Keith, Max beanDerivative: 18.0, Index Time: 318
Roast Name: #102 Columbia Huila Double Ferment -  hot night, Max beanDerivative: 21.1, Index Time: 264
Roast Name: #103 - Ethiopia Hambela Dabaye - 211 drop, Max beanDerivative: 19.7, Index Time: 281
Roast Name: #104 Colombia Huila Double Ferment , Max beanDerivative: 20.8, Index Time: 290
Roast Name: #105 Ethiopia yirg worka natural from rob 5lb , Max beanDerivative: 18.3, Index Time: 324
Roast Name: #106 Ethiopia Worka Natural Yirg, Max beanDerivative: 18.9, Index Time: 287
Roast Name: #107 Ethiopia Yirg Worka Natty - Good bridge between last two, Max beanDerivative: 22.0, Index Time: 318
Roast Name: #108 Guatemala Huehuetenago Coffee Kids, Max beanDerivative: 25.7, Index Time: 4
Roast Name: #109 Co

In [31]:
## SHIP THIS to main

# setting 20-25 window size
window_size = 20
new_df = pd.DataFrame()
new_df['peakROR'] = curve_df.groupby('roastName')['beanDerivative'].rolling(window=window_size, center=True).mean().groupby(level=0).max()

# Define a custom function to get the indexTime associated with the peak ROR
def get_max_index_and_time(group):
    rolling_mean = (
        group['beanDerivative']
        .rolling(window=window_size, center=True)
        .mean()
    )
    max_index_time = rolling_mean.nlargest(1).idxmin()
    return pd.Series({
        'roastName': group['roastName'].iloc[0],  # Take the roastName value from the group
        'maxROR25_indexTime': group.loc[max_index_time, 'indexTime']
    })

# Apply the custom function to each group
maxROR_info = curve_df.groupby('roastName').apply(get_max_index_and_time).reset_index(drop=True)

# Merge new_df with maxROR_info on 'roastName'
new_df = pd.merge(new_df, maxROR_info, on='roastName', how='left')
new_df['peakRORTime'] = new_df['maxROR25_indexTime']/2
new_df = new_df.reset_index(drop=True)
#merge new_df into point_df based on roastName
point_df = point_df.merge(new_df[['roastName', 'peakROR', 'peakRORTime']], on='roastName', how='left')




,roastName,peakROR,maxROR25_indexTime,peakRORTime
0,# 25 Kenya Mix 350/140,18.555,284,142.0
1,#100 Columbia Hulia double ferment - heater er...,17.615,364,182.0
2,#101 El Salvador Siberia Washed w/ Keith,17.315,315,157.5
3,#102 Columbia Huila Double Ferment - hot night,20.030,264,132.0
4,#103 - Ethiopia Hambela Dabaye - 211 drop,19.320,284,142.0
...,...,...,...,...
15,#113 Ethiopia Mix Refisia and Hambela Dabaye,18.720,295,147.5
16,#114 Guatemalan Huehuetenango Coffee Kids,20.230,289,144.5
17,#115 Guatemala Huehuetenango Coffee Kids 650g/...,20.035,320,160.0
18,#116 Colombia Huila Double Ferment 450g,21.645,294,147.0


In [32]:
#merge new_df into point_df based on roastName
point_df = point_df.merge(new_df[['roastName', 'peakROR', 'peakRORTime']], on='roastName', how='left')
point_df

,roastName,dateTime,beanChargeTemperature,beanDropTemperature,drumChargeTemperature,drumDropTemperature,preheatTemperature,weightGreen,weightRoasted,weightLostPercent,...,deltaIBTS-BT-atDrop,yellowingPhaseTime,browningPhaseTime,developmentTime,RoR-yellowing-est,RoR-browning-est,RoR-development-est,RoR-fullRoast-est,peakROR,peakRORTime
0,First Post-seasoning,1595642007716,125.90,187.60,237.70,204.60,240,370.0,0.0,NaN,...,17.00,5.458333,1.033333,539.666667,12.586260,38.612903,1.0,0.198294,19.210,164.0
1,2nd Ethopia Extended Dev,1595983767878,150.10,189.80,244.90,207.60,245,380.0,336.0,11.578947,...,17.80,2.466667,2.950000,476.666667,25.743243,12.169492,1.0,0.220086,19.815,169.0
2,First BBP attempt - CItyish,1596160358079,147.80,202.60,245.10,216.10,245,490.0,425.0,13.265306,...,13.50,2.633333,3.466667,506.016667,27.037975,12.000000,1.0,0.238813,19.170,157.0
3,Ethiopia #4 w/ marcel,1596417079393,150.00,199.80,245.30,208.60,245,490.0,0.0,NaN,...,8.80,2.775000,3.533333,552.733333,23.711712,11.179245,1.0,0.195692,17.285,161.0
4,Ethiopian 5th - Hot Start,1596498641198,158.70,198.90,247.30,211.00,245,490.0,427.0,12.857143,...,12.10,2.408333,3.150000,509.500000,26.283737,11.841270,1.0,0.212209,17.700,130.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,#134 - Crown Jewel Ethiopian,1692549509954,164.50,215.30,245.60,215.80,245,750.0,647.0,13.733333,...,0.50,4.125000,3.683333,657.183333,18.666667,9.828054,1.0,0.192183,17.170,161.5
128,#136 Crown Jewel Eth,1696604639386,157.52,209.30,241.71,212.29,240,650.0,567.0,12.769231,...,2.99,2.958333,2.991667,516.108333,25.680000,11.919777,1.0,0.236104,22.115,110.0
129,#137 - Crown Jewel Ethiopian WW,1696606297831,161.21,207.29,241.71,209.66,240,650.0,567.8,12.646154,...,2.37,3.300000,3.416667,542.283333,22.245455,10.922927,1.0,0.215064,19.378,150.0
130,#138 - Ethiopia Wenagno Wolde Hirbe Natural - ...,1702252851618,0.00,0.00,0.00,0.00,250,500.0,437.0,12.600000,...,0.00,2.125000,2.841667,492.075000,30.884706,12.640469,1.0,-0.199923,NaN,NaN


In [48]:
# Determine the max ROR for each roastName using an average moving window of 50 points (25 seconds)
sampleRate = 2
point_df['peakRoR'] = curve_df.groupby('roastName')['beanDerivative'].rolling(20, center=True).mean().groupby(level=1).max().reset_index(drop=True)
window_size = 50
roll_max = curve_df.groupby('roastName')['beanDerivative'].apply(lambda x: x.rolling(window=window_size, center=True).mean().idxmax() / 60 / sampleRate).reset_index(drop=True)
point_df['peakRoRTime'] = roll_max
display (point_df[['roastName','peakRoR','peakRoRTime']].tail(20))

,roastName,peakRoR,peakRoRTime
112,#119 Crown Jewel Ethiopia Organic Wenago Wolde...,1.030,905.350000
113,#120 Crown Jewel Ethiopia Organic Wenago Wolde...,1.395,920.516667
114,#121 - Rwanda 1st with Jim,1.760,932.958333
115,#122 - Rwanda 2nd,2.120,945.266667
116,#123 Crown Jewel Ethiopia Wengo,2.485,955.275000
117,#124 Crown Ethiopia 2,2.835,966.275000
118,#125 - Ethiopia Crown Jewel Natural Process 4-16,3.215,976.225000
119,#126 - Ethiopia Crown Jewels Natural Wengo,3.605,987.141667
120,#127 - Colombia Santa Barbra Natural - 1st & o...,3.995,997.641667
121,#128 - Colombia Santa Barbra Natural - Power o...,4.385,56.050000


In [50]:
# Calculate the peak RoR for each roastName in curve_df
peakRoR = curve_df.groupby('roastName')['beanDerivative'].rolling(20, center=True).mean().groupby(level=0).max()

# Reset the index of peakRoR
peakRoR = peakRoR.reset_index()
display (peakR0R)


,roastName,peakRoR,peakRoRTime
112,#119 Crown Jewel Ethiopia Organic Wenago Wolde...,1.030,905.350000
113,#120 Crown Jewel Ethiopia Organic Wenago Wolde...,1.395,920.516667
114,#121 - Rwanda 1st with Jim,1.760,932.958333
115,#122 - Rwanda 2nd,2.120,945.266667
116,#123 Crown Jewel Ethiopia Wengo,2.485,955.275000
117,#124 Crown Ethiopia 2,2.835,966.275000
118,#125 - Ethiopia Crown Jewel Natural Process 4-16,3.215,976.225000
119,#126 - Ethiopia Crown Jewels Natural Wengo,3.605,987.141667
120,#127 - Colombia Santa Barbra Natural - 1st & o...,3.995,997.641667
121,#128 - Colombia Santa Barbra Natural - Power o...,4.385,56.050000


In [ ]:

# Merge peakRoR with point_df on 'roastName'
point_df = pd.merge(point_df, peakRoR, on='roastName', how='left')
display (point_df[['roastName','peakRoR','peakRoRTime']].tail(20))

In [ ]:
# Calculate the rolling median of 'beanDerivative' with a window size of 50
rolling_median = curve_df['beanDerivative'].rolling(window=50).median()

# Find the maximum value of the rolling median for each 'roastName'
point_df['peakRoR'] = curve_df.groupby('roastName')[rolling_median].transform('max')

# Find the time at which the maximum RoR occurs
curve_df['peakRoRTime'] = curve_df.groupby('roastName')[rolling_median].transform('idxmax')
curve_df['peakRoRTime'] = curve_df['peakRoRTime'] / (60 * sample_rate)

display (point_df[['roastName','peakRoR','peakRoRTime']].tail(20))

In [7]:
curve_df

,beanTemperature,drumTemperature,beanDerivative,ibtsDerivative,roastName,indexTime,softwareVersion,ibts2ndDerivative
0,125.90,237.70,18.70,0.00,First Post-seasoning,0,NaN,0.00
1,126.10,237.60,17.60,0.00,First Post-seasoning,1,NaN,0.00
2,126.20,237.90,16.10,0.00,First Post-seasoning,2,NaN,0.00
3,126.30,238.00,14.30,0.00,First Post-seasoning,3,NaN,0.00
4,126.40,237.30,12.80,0.00,First Post-seasoning,4,NaN,0.00
...,...,...,...,...,...,...,...,...
162826,207.00,209.44,9.95,4.91,#137 - Crown Jewel Ethiopian WW,1099,FE:v1.113.0;BE:v1.40.12;Comms:v1.10.56,0.56
162827,207.08,209.44,9.03,4.91,#137 - Crown Jewel Ethiopian WW,1100,FE:v1.113.0;BE:v1.40.12;Comms:v1.10.56,0.00
162828,207.15,209.54,8.40,5.69,#137 - Crown Jewel Ethiopian WW,1101,FE:v1.113.0;BE:v1.40.12;Comms:v1.10.56,0.78
162829,207.21,209.54,8.36,5.69,#137 - Crown Jewel Ethiopian WW,1102,FE:v1.113.0;BE:v1.40.12;Comms:v1.10.56,0.00
